In [1]:
import gzip
import ast
import json
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

vamos a traer el primer Archivo JSON. llamado output_steam_games

In [2]:


# Leer el archivo JSON línea por línea y cargar cada objeto JSON en la lista 'data'
data = []
with open('D:/USUARIO/Desktop/datos/output_steam_games.json', 'r') as file:
    for line in file:
        try:
            obj = json.loads(line)
            data.append(obj)
        except json.JSONDecodeError:
            print(f"Error al decodificar JSON en la línea: {line}")

vamos almacenar el dataframe con el nombre de steam_games_df

In [3]:
steam_games_df =pd.DataFrame(data)

In [4]:
filas_nulas = steam_games_df.isnull().all(axis=1).sum()
print(filas_nulas)

88310


Eliminamos las filas nulas de nuesto steam_games_df

In [5]:
steam_games_df= steam_games_df.dropna()


Vamos a trabajar con el archivo JSON llamado australian_user_reviews

In [6]:
with open('D:/USUARIO/Desktop/datos/australian_user_reviews.json', 'r', encoding='utf-8') as file:
    data = [eval(line) for line in file]

user_reviews = pd.DataFrame(data)

Luego de tener el archivo requerimos hacer el explode, esto con la finalidad de desanidar la columna reviews, luego la normalizamos para que tenga cada columna. para el final hacemos el concat de los dos dataset y eliminamos la comulna reviewa, y nos queda el nuevo dataSet

In [7]:
user_reviews_explored = user_reviews.explode('reviews')
user_reviews_normalized = pd.json_normalize(user_reviews_explored['reviews'])
user_reviews_explored.reset_index(drop=True, inplace=True)
user_reviews_normalized.reset_index(drop=True, inplace=True)
user_reviews_final = pd.concat([user_reviews_explored.drop('reviews',axis=1), user_reviews_normalized], axis=1)

Hacemos el mismo proceso para el archivo llamado australian_users_items. lo extraemos del archivo JSON

In [8]:
with open('D:/USUARIO/Desktop/datos/australian_users_items.json', 'r', encoding='utf-8') as file:
    data2 = [eval(line) for line in file]

user_items = pd.DataFrame(data2)

procedemos a desanidar el dataframe en la columna items

In [9]:
user_items_explored = user_items.explode('items')
user_items_normalized = pd.json_normalize(user_items_explored['items'])
user_items_explored.reset_index(drop=True, inplace=True)
user_items_normalized.reset_index(drop=True, inplace=True)
user_items_final = pd.concat([user_items_explored.drop('items',axis=1), user_items_normalized], axis=1)

In [10]:
user_items_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


Vamos a tomar este dataframe  para hacer un groupby, esto con el fin de tener las horas acumuladas  (playtime_forever)  por cada item_id.

In [11]:
df_agrupado = user_items_final.groupby(['item_id'])['playtime_forever'].sum().reset_index().rename(columns={'playtime_forever':'suma_playtime_forever'})


In [12]:
df_agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10978 entries, 0 to 10977
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   item_id                10978 non-null  object 
 1   suma_playtime_forever  10978 non-null  float64
dtypes: float64(1), object(1)
memory usage: 171.7+ KB


Vamos a trabajar con el dataframe  user_review_final. primero vamos a crear la columna 'sentiment-analysis' esto lo vamos hacer con la libreria Nnltk que sirve para esudio de analisis de sentimiento, de acuerdo a los   valores que se veran reflejados en la columna, haremos luegola conversión cmo lo piden en el enunciado.

In [13]:
sid = SentimentIntensityAnalyzer()

In [14]:
user_reviews_final['sentiment-analysis'] = user_reviews_final['review'].apply(lambda x: sid.polarity_scores(str(x))['compound'] if isinstance(x, str) else 0)

aca vamos hacer el cambio de acuerdo al enunciado: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews

In [15]:
user_reviews_final.loc[user_reviews_final['sentiment-analysis'] > 0, 'sentiment-analysis'] = 2
user_reviews_final.loc[user_reviews_final['sentiment-analysis'] == 0, 'sentiment-analysis'] = 1
user_reviews_final.loc[user_reviews_final['sentiment-analysis'] < 0, 'sentiment-analysis'] = 0

In [16]:
user_reviews_final.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment-analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2.0
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2.0
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2.0


Procedemos a eliminar la columna review.

In [17]:
user_reviews_final = user_reviews_final.drop('review',axis=1)

In [18]:
user_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             59333 non-null  object 
 1   user_url            59333 non-null  object 
 2   funny               59305 non-null  object 
 3   posted              59305 non-null  object 
 4   last_edited         59305 non-null  object 
 5   item_id             59305 non-null  object 
 6   helpful             59305 non-null  object 
 7   recommend           59305 non-null  object 
 8   sentiment-analysis  59333 non-null  float64
dtypes: float64(1), object(8)
memory usage: 4.1+ MB


In [19]:
conteo_valores = user_reviews_final['sentiment-analysis'].value_counts()
print(conteo_valores)

sentiment-analysis
2.0    37657
1.0    11844
0.0     9832
Name: count, dtype: int64


Procedemos a unir el user_reviews_final con df_agrupado haciendo un merge con la columna en comun 'item_id'

In [20]:
df_merge1 = pd.merge(df_agrupado,user_reviews_final,on='item_id')

ahora para concluir debemos hacer la union con el dataframe faltante llamado steam_games_df, pero antes debemos cmabiar el nombre de  'id'po 'item_id'

In [21]:
steam_games_df = steam_games_df.rename(columns={'id': 'item_id'})

ahora si vamos hacer el merge. de steam_games_df con df:merge1

In [22]:
consolidado_df = pd.merge(df_merge1 ,steam_games_df,on='item_id')

In [23]:
consolidado_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44680 entries, 0 to 44679
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   item_id                44680 non-null  object 
 1   suma_playtime_forever  44680 non-null  float64
 2   user_id                44680 non-null  object 
 3   user_url               44680 non-null  object 
 4   funny                  44680 non-null  object 
 5   posted                 44680 non-null  object 
 6   last_edited            44680 non-null  object 
 7   helpful                44680 non-null  object 
 8   recommend              44680 non-null  object 
 9   sentiment-analysis     44680 non-null  float64
 10  publisher              44680 non-null  object 
 11  genres                 44680 non-null  object 
 12  app_name               44680 non-null  object 
 13  title                  44680 non-null  object 
 14  url                    44680 non-null  object 
 15  re

In [24]:
consolidado_df.head()

,item_id,suma_playtime_forever,user_id,user_url,funny,posted,last_edited,helpful,recommend,sentiment-analysis,...,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,developer
0,10,17386015.0,Bennysaputra,http://steamcommunity.com/id/Bennysaputra,,"Posted August 1, 2015.",,No ratings yet,True,2.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve
1,10,17386015.0,Monsta45,http://steamcommunity.com/id/Monsta45,,"Posted April 26, 2015.",,No ratings yet,True,1.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve
2,10,17386015.0,76561198040188061,http://steamcommunity.com/profiles/76561198040...,,"Posted May 18, 2011.",,0 of 2 people (0%) found this review helpful,True,2.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve
3,10,17386015.0,mayshowganmore,http://steamcommunity.com/id/mayshowganmore,,"Posted January 22, 2014.",,0 of 2 people (0%) found this review helpful,True,2.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve
4,10,17386015.0,BestinTheWorldThund3r,http://steamcommunity.com/id/BestinTheWorldThu...,,"Posted March 19, 2014.",,0 of 2 people (0%) found this review helpful,True,2.0,...,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,Valve


In [25]:
consolidado_df.to_csv('padre_df1.csv',index = False)

A partir de estos procesos vamos  a crear cada uno de los dataset por separado para cada una de las funciones.

In [26]:
df_1= pd.read_csv('padre_df1.csv')


Vamos a extraer las columnas: genres,suma_player_forever, release_date

In [27]:
df_1_1= df_1.loc[:,['genres','suma_playtime_forever','release_date','user_id']]

In [28]:
df_1_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44680 entries, 0 to 44679
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   genres                 44680 non-null  object 
 1   suma_playtime_forever  44680 non-null  float64
 2   release_date           44680 non-null  object 
 3   user_id                44680 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.4+ MB


In [29]:


def convertir_a_lista(elemento):
    try:
        return ast.literal_eval(elemento)
    except (SyntaxError, ValueError):
        return elemento

df_1_1['genres'] = df_1_1['genres'].apply(convertir_a_lista)


In [30]:
df_1_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44680 entries, 0 to 44679
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   genres                 44680 non-null  object 
 1   suma_playtime_forever  44680 non-null  float64
 2   release_date           44680 non-null  object 
 3   user_id                44680 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.4+ MB


In [31]:
df_1_1.head(16)

,genres,suma_playtime_forever,release_date,user_id
0,[Action],17386015.0,2000-11-01,Bennysaputra
1,[Action],17386015.0,2000-11-01,Monsta45
2,[Action],17386015.0,2000-11-01,76561198040188061
3,[Action],17386015.0,2000-11-01,mayshowganmore
4,[Action],17386015.0,2000-11-01,BestinTheWorldThund3r
5,[Action],17386015.0,2000-11-01,apex124
6,[Action],17386015.0,2000-11-01,76561198072207162
7,[Action],17386015.0,2000-11-01,timtam012
8,[Action],17386015.0,2000-11-01,chewyi
9,[Action],17386015.0,2000-11-01,Bigzy


In [32]:
df_1_1= df_1_1.explode('genres')

In [33]:
df_1_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113452 entries, 0 to 44679
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   genres                 113452 non-null  object 
 1   suma_playtime_forever  113452 non-null  float64
 2   release_date           113452 non-null  object 
 3   user_id                113452 non-null  object 
dtypes: float64(1), object(3)
memory usage: 4.3+ MB


In [34]:
df_1_1.head()


,genres,suma_playtime_forever,release_date,user_id
0,Action,17386015.0,2000-11-01,Bennysaputra
1,Action,17386015.0,2000-11-01,Monsta45
2,Action,17386015.0,2000-11-01,76561198040188061
3,Action,17386015.0,2000-11-01,mayshowganmore
4,Action,17386015.0,2000-11-01,BestinTheWorldThund3r


vamos arreglar la fecha

In [35]:
df_1_1['release_date']=pd.to_datetime(df_1_1['release_date'],errors='coerce')



In [36]:
df_1_1['release_date']=df_1_1['release_date'].dt.year


In [37]:
moda= df_1_1['release_date'].mode()[0]
print(moda)

2013.0


In [38]:
df_1_1['release_date']= df_1_1['release_date'].fillna(moda)

In [39]:
valores_nulos= df_1_1['release_date'].isnull().sum()
print(valores_nulos)

0


In [ ]:
df_1_1.head()

,genres,suma_playtime_forever,release_date,user_id
0,Action,17386015.0,2000.0,Bennysaputra
1,Action,17386015.0,2000.0,Monsta45
2,Action,17386015.0,2000.0,76561198040188061
3,Action,17386015.0,2000.0,mayshowganmore
4,Action,17386015.0,2000.0,BestinTheWorldThund3r


In [40]:
df_1_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113452 entries, 0 to 44679
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   genres                 113452 non-null  object 
 1   suma_playtime_forever  113452 non-null  float64
 2   release_date           113452 non-null  float64
 3   user_id                113452 non-null  object 
dtypes: float64(2), object(2)
memory usage: 4.3+ MB


In [41]:
df_1_1.head()

,genres,suma_playtime_forever,release_date,user_id
0,Action,17386015.0,2000.0,Bennysaputra
1,Action,17386015.0,2000.0,Monsta45
2,Action,17386015.0,2000.0,76561198040188061
3,Action,17386015.0,2000.0,mayshowganmore
4,Action,17386015.0,2000.0,BestinTheWorldThund3r


In [42]:
df_1_1['release_date'] = df_1_1['release_date'].astype(int)

In [43]:
df_1_1.head()

,genres,suma_playtime_forever,release_date,user_id
0,Action,17386015.0,2000,Bennysaputra
1,Action,17386015.0,2000,Monsta45
2,Action,17386015.0,2000,76561198040188061
3,Action,17386015.0,2000,mayshowganmore
4,Action,17386015.0,2000,BestinTheWorldThund3r


aca queda consolidado  nuestro dataset  para crear la función numero 1 y la numero 2

In [44]:
df_1_1.to_csv('df_funcion3.csv',index = False)

**Vamos a Crear el dataset para realizar la función numero 3 y 4. para esto se reuqieren las siguientes columnas del dataset padre: developer, user_id, app_name, recommend y posted.**


In [45]:
df_4 = df_1.loc[:,['developer' ,'user_id','app_name','recommend','posted']]

In [46]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44680 entries, 0 to 44679
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   developer  44680 non-null  object
 1   user_id    44680 non-null  object
 2   app_name   44680 non-null  object
 3   recommend  44680 non-null  bool  
 4   posted     44680 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.4+ MB


In [47]:
df_4.head()

,developer,user_id,app_name,recommend,posted
0,Valve,Bennysaputra,Counter-Strike,True,"Posted August 1, 2015."
1,Valve,Monsta45,Counter-Strike,True,"Posted April 26, 2015."
2,Valve,76561198040188061,Counter-Strike,True,"Posted May 18, 2011."
3,Valve,mayshowganmore,Counter-Strike,True,"Posted January 22, 2014."
4,Valve,BestinTheWorldThund3r,Counter-Strike,True,"Posted March 19, 2014."


In [ ]:
De esta manera concluimos el procedimiento de construir el data set  para la cuarta funsión.

In [ ]:
df_4.to_csv('df_funcion4.csv',index = False)

**Procedemos a crear el quinto dataset para porder hacer  la consulta de la  quinta función.**

In [ ]:
df_5 = df_1.loc[:,['developer','sentiment-analysis']]

In [ ]:
df_5.head()

,developer,sentiment-analysis
0,Valve,2.0
1,Valve,1.0
2,Valve,2.0
3,Valve,2.0
4,Valve,2.0


In [ ]:
df_5.to_csv('df_funcion5.csv',index = False)


**Vamos a crear el dataSet para la consulta de Machine Learning.**

In [ ]:
df_6= df_1.loc[:,['user_id','title','tags','genres']]

In [ ]:
df_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44680 entries, 0 to 44679
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  44680 non-null  object
 1   title    44680 non-null  object
 2   tags     44680 non-null  object
 3   genres   44680 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [ ]:
df_6.head()

,user_id,title,tags,genres
0,Bennysaputra,Counter-Strike,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",['Action']
1,Monsta45,Counter-Strike,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",['Action']
2,76561198040188061,Counter-Strike,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",['Action']
3,mayshowganmore,Counter-Strike,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",['Action']
4,BestinTheWorldThund3r,Counter-Strike,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",['Action']


vamos a desanidar la columna genres.

In [ ]:
def convertir_a_lista(elemento):
    try:
        return ast.literal_eval(elemento)
    except (SyntaxError, ValueError):
        return elemento

df_6['genres'] = df_5['genres'].apply(convertir_a_lista)


ahora hacemos el explode

In [ ]:
df_6= df_6.explode('genres')

In [ ]:
df_6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113452 entries, 0 to 44679
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  113452 non-null  object
 1   title    113452 non-null  object
 2   tags     113452 non-null  object
 3   genres   113452 non-null  object
dtypes: object(4)
memory usage: 4.3+ MB


ahora hacemos lo mismo para la columna tags

In [ ]:
def convertir_a_lista(elemento):
    try:
        return ast.literal_eval(elemento)
    except (SyntaxError, ValueError):
        return elemento

df_6['tags'] = df_6['tags'].apply(convertir_a_lista)

In [ ]:
df_6= df_6.explode('tags')

In [ ]:
df_6.head()

,user_id,title,tags,genres
0,Bennysaputra,Counter-Strike,Action,Action
0,Bennysaputra,Counter-Strike,FPS,Action
0,Bennysaputra,Counter-Strike,Multiplayer,Action
0,Bennysaputra,Counter-Strike,Shooter,Action
0,Bennysaputra,Counter-Strike,Classic,Action


In [ ]:
df_6.reset_index(drop=True, inplace=True)

In [ ]:
df_6.head()

,user_id,title,tags,genres
0,Bennysaputra,Counter-Strike,Action,Action
1,Bennysaputra,Counter-Strike,FPS,Action
2,Bennysaputra,Counter-Strike,Multiplayer,Action
3,Bennysaputra,Counter-Strike,Shooter,Action
4,Bennysaputra,Counter-Strike,Classic,Action


Aca Vamos a quitar los duplicados para poder trabajar con un dataset mas compacto y manejable para poder aplicar la similitud de coseno entre los juegos, esto con el fin de poder correr la API y en render, ya que un tamaño muy grande del dataset sera algo dificil de poder realizar la consulta.

In [ ]:
df_sin_duplicados = df_6.drop_duplicates(subset=["title"])
df_sin_duplicados.head()

,user_id,title,tags,genres
0,Bennysaputra,Counter-Strike,Action,Action
1140,76561198073508075,Call of Duty: World at War,Zombies,Action
2180,Shemhamphorasch,TimeShift™,Action,Action
2206,tsunamitad,3D Ultra™ Minigolf Adventures,Sports,Sports
2210,TomoeMamiewe,Prototype™,Action,Action


In [ ]:
df_sin_duplicados.reset_index(drop=True, inplace=True)

In [ ]:
df_sin_duplicados.to_csv('df_funcion6.csv')